In [8]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

In [9]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

In [14]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['id_persona','FECHA_FALLECIMIENTO', 'EDAD_DECLARADA', 'SEXO', 'DEPARTAMENTO']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})

In [15]:
# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'EDAD_DECLARADA': 'edad',
                         'SEXO': 'sexo', 'DEPARTAMENTO': 'departamento'}, inplace=True)

df_fal.drop(df_fal[df_fal.sexo == '.'].index, inplace = True)
df_fal.drop(df_fal[df_fal.edad == -9].index, inplace = True)  # un -9 en edad

df_fal.loc[:,'fecha_fallecimiento'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')  # Transformar a formato fecha
df_fal['edad_cat'] = edad_cat(df_fal, 'edad')  # Convertir edad en categoría
df_fal['sexo'] = df_fal['sexo'].replace(dic_sex).astype('int8')  # Reemplazar sexo con un code
df_fal['departamento'] = df_fal['departamento'].map(dic_dep).astype('int8')  # Codificar variables

fecha_min = df_fal['fecha_fallecimiento'].min()  # Agregar la diferencia en días desde el primer caso
df_fal['time_days'] = df_fal['fecha_fallecimiento'].apply(lambda row : days_between(row, fecha_min)+1)  # Agregar el num de días
# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha_fallecimiento'].apply(lambda date : Week.fromdate(date) \
                                                                                    .weektuple()) \
                                                                                    .tolist()
# Reorder
df_fal = df_fal[['id_persona', 'fecha_fallecimiento','year', 'epi_week', 'time_days', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_fallecimiento']) \
         .reset_index(drop = True)

In [16]:
df = dd.from_pandas(df_fal, npartitions=3)
del df_fal
df.head(10)

,id_persona,fecha_fallecimiento,year,epi_week,time_days,sexo,edad,edad_cat,departamento
0,36527082.0,2020-03-03,2020,10,1,1,63,5,22
1,36401995.0,2020-03-04,2020,10,2,1,85,7,22
2,36853977.0,2020-03-07,2020,10,5,2,24,1,16
3,NaN,2020-03-11,2020,11,9,2,78,6,20
4,36614476.0,2020-03-14,2020,11,12,1,4,0,15
5,NaN,2020-03-14,2020,11,12,1,8,0,16
6,36437890.0,2020-03-16,2020,12,14,1,73,6,15
7,36645051.0,2020-03-16,2020,12,14,2,88,7,15
8,36731113.0,2020-03-16,2020,12,14,1,57,4,15
9,36769383.0,2020-03-16,2020,12,14,1,33,2,15


In [17]:
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8',
                                                                      'id_vacuna': 'int8', 'id_persona': 'float64'})

ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,17289500.0,1,20210623,4
1,19180697.0,1,20211210,6
2,19374477.0,3,20211221,5
3,19152824.0,2,20210619,4
4,23501162.0,3,20211211,5


In [18]:
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')
del df_dosis_3, ddf_vac

In [19]:
df = df.compute()

In [20]:
# Variable "vacunado" si recibe las 2 dosis
df['vacunado'] = np.where(df['dosis_2'] == 1, 1, 0).astype('int8')

df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [21]:
df = df[['id_persona', 'fecha_fallecimiento', 'year', 'epi_week', 'time_days',
       'sexo', 'edad', 'edad_cat', 'departamento', 'vacunado', 'dosis_1', 'fecha_vacuna_1',
       'id_vacuna_1', 'dosis_2', 'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3',
       'fecha_vacuna_3', 'id_vacuna_3']]

In [22]:
df = df.sort_values(by = 'fecha_fallecimiento').reset_index(drop = True)

In [23]:
df

,id_persona,fecha_fallecimiento,year,epi_week,time_days,sexo,edad,edad_cat,departamento,vacunado,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,36527082.0,2020-03-03,2020,10,1,1,63,5,22,0,0,NaN,0,0,NaN,0,0,NaN,0
1,36401995.0,2020-03-04,2020,10,2,1,85,7,22,0,0,NaN,0,0,NaN,0,0,NaN,0
2,36853977.0,2020-03-07,2020,10,5,2,24,1,16,0,0,NaN,0,0,NaN,0,0,NaN,0
3,NaN,2020-03-11,2020,11,9,2,78,6,20,0,0,NaN,0,0,NaN,0,0,NaN,0
4,36614476.0,2020-03-14,2020,11,12,1,4,0,15,0,0,NaN,0,0,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202971,22956169.0,2022-01-06,2022,1,675,1,86,7,2,1,1,20210601.0,5,1,20210816.0,5,0,NaN,0
202972,15072064.0,2022-01-06,2022,1,675,2,70,6,11,1,1,20210222.0,6,1,20210315.0,6,1,20211019.0,5
202973,2181753.0,2022-01-06,2022,1,675,1,93,7,15,1,1,20210508.0,5,1,20210529.0,5,0,NaN,0
202974,18627612.0,2022-01-06,2022,1,675,1,88,7,2,0,1,20210630.0,5,0,NaN,0,0,NaN,0


In [24]:
df.to_csv('../Data/DP6_covid19-peru_info-fallecidos.csv', index = False)